![LipPike](https://upload.wikimedia.org/wikipedia/commons/3/32/Lip_Pike_Baseball.jpg "Lip Pike had Style")

### Back in the Good Ole Days of Base Ball...

In [1]:
cd ..

/Users/johnphillips/Desktop/DSI-Class-Stuff/BaseBall


In [2]:
%matplotlib inline

In [3]:
%run __init__.py

In [4]:
# Start with reading the data sets from the pickles

teams = pd.read_pickle('data/teams.p') # Full Data
teams_num = pd.read_pickle('data/teams_num.p') # Numeric Features
new_reduced = pd.read_pickle('data/new_reduced.p') # Reduced from PCA
salaries = pd.read_pickle('data/salaries.p') # Team Salaries 1985 - 2016
rockies = pd.read_pickle('data/rockies.p') # 2017 Rockies Team Stats

### Can we predict which team would win the World Series before 1903?

In [5]:
# Start with separating old_teams:
old_teams = teams[(teams['yearID'] < 1903)]
old_teams_num = old_teams.select_dtypes(exclude=['object', 'category'])

# Had to also drop Wins, Losses, and WinP since those are direct indicators.
# And they are not part of a players individual stats.
X_old = old_teams_num.drop(['WSWin', 'LgWin', 'W', 'L', 'WinP'], axis=1) 
y_old = old_teams_num['LgWin']

In [6]:
# Start with separating new_teams
new_teams = teams[(teams['yearID'] > 1903)]
new_teams_num = new_teams.select_dtypes(exclude=['object', 'category'])

# Had to also drop Wins, Losses, and WinP since those are direct indicators.
# And they are not part of a players individual stats.
X_new = new_teams_num.drop(['WSWin', 'LgWin', 'W', 'L', 'WinP'], axis=1) 
y_new = new_teams_num['WSWin']

In [7]:
# Had to also drop Wins, Losses, and WinP since those are direct indicators.
# And they are not part of a players individual stats.
X = teams_num.drop(['WSWin', 'LgWin', 'W', 'L', 'WinP'], axis=1) 

# Make a y2 based upon Winning Leage Series, for possible second model.
y = teams_num['WSWin']

### Deep Learning with Keras

In [8]:
import keras
from keras.layers import Input, Conv2D, MaxPool2D, Dropout, Activation, Dense, Flatten
from keras.models import Sequential
from keras.activations import relu, softmax
from keras.utils import to_categorical

/Users/johnphillips/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
Couldn't import dot_parser, loading of dot files will not be possible.


In [9]:
# Convert the target to categorical: target
target = to_categorical(y)
predictors = X.as_matrix()
n_cols = predictors.shape[1]

In [10]:
# Specify, compile, and fit the model
model = Sequential()
model.add(Dense(264, activation='relu', input_shape = (n_cols,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
model.fit(predictors, target, epochs=15)


Epoch 1/15
2835/2835 [==============================] - ETA: 0s - loss: 0.7986 - acc: 0.950 - 1s 258us/step - loss: 0.7926 - acc: 0.9506
Epoch 2/15
2835/2835 [==============================] - 0s 76us/step - loss: 0.6652 - acc: 0.9587
Epoch 3/15
2835/2835 [==============================] - 0s 73us/step - loss: 0.6652 - acc: 0.9587
Epoch 4/15
2835/2835 [==============================] - 0s 78us/step - loss: 0.6652 - acc: 0.9587
Epoch 5/15
2835/2835 [==============================] - 0s 74us/step - loss: 0.6652 - acc: 0.9587
Epoch 6/15
2835/2835 [==============================] - 0s 75us/step - loss: 0.6652 - acc: 0.9587
Epoch 7/15
2835/2835 [==============================] - 0s 75us/step - loss: 0.6652 - acc: 0.9587
Epoch 8/15
2835/2835 [==============================] - 0s 76us/step - loss: 0.6652 - acc: 0.9587
Epoch 9/15
2835/2835 [==============================] - 0s 74us/step - loss: 0.6652 - acc: 0.9587
Epoch 10/15
2835/2835 [==============================] - 0s 79us/step - loss: 0

In [11]:
from keras.models import load_model
model.save('model_file.h5')
model_1 = load_model('model_file.h5')
predictions = model_1.predict(X_new)
proba_win = predictions[:, 1]

ValueError: could not broadcast input array from shape (36,1) into shape (36)

In [ ]:
model_1.summary()

In [ ]:
# Save the number of columns in predictors: n_cols
n_cols = predictors.shape[1]
input_shape = (n_cols,)

# Specify the model
model = Sequential()
model.add(Dense(100, activation='relu', input_shape = input_shape))
model.add(Dense(100, activation='relu'))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model
hist = model.fit(predictors, target, validation_split=0.3)

In [ ]:
# Import EarlyStopping
from keras.callbacks import EarlyStopping

# Save the number of columns in predictors: n_cols
n_cols = predictors.shape[1]
input_shape = (n_cols,)

# Specify the model
model = Sequential()
model.add(Dense(200, activation='relu', input_shape = input_shape))
model.add(Dense(250, activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define early_stopping_monitor
early_stopping_monitor = EarlyStopping(patience=3)

# Fit the model
model.fit(predictors, target, epochs=30, validation_split=0.3, callbacks=[early_stopping_monitor])


In [12]:
# Define early_stopping_monitor
early_stopping_monitor = EarlyStopping(patience=3)

# Create the new model: model_2
model_2 = Sequential()

# Add the first and second layers
model_2.add(Dense(350, activation='relu', input_shape=input_shape))
model_2.add(Dense(200, activation='relu'))
model_2.add(Dense(2, activation='softmax'))

# Compile model_2
model_2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Fit model_1
model_1_training = model.fit(predictors, target, epochs=15, validation_split=0.3, callbacks=[early_stopping_monitor], verbose=False)

# Fit model_2
model_2_training = model_2.fit(predictors, target, epochs=15, validation_split=0.3, callbacks=[early_stopping_monitor], verbose=False)

# Create the plot
plt.plot(model_1_training.history['val_loss'], 'r', model_2_training.history['val_loss'], 'b')
plt.xlabel('Epochs')
plt.ylabel('Validation score')
plt.show()

NameError: name 'EarlyStopping' is not defined

### Back to the Regression Problem

In [13]:
# Start with separating old_teams:
old_teams = teams[(teams['yearID'] < 1903)]
old_teams_num = old_teams.select_dtypes(exclude=['object', 'category'])

# Had to also drop Wins, Losses, and WinP since those are direct indicators.
# And they are not part of a players individual stats.
X_old = old_teams_num.drop(['WSWin', 'LgWin', 'W', 'L', 'WinP'], axis=1) 
y_old = old_teams_num['WinP']

In [14]:
# Had to also drop Wins, Losses, and WinP since those are direct indicators.
# And they are not part of a players individual stats.
X = teams_num.drop(['WSWin', 'LgWin', 'W', 'L', 'WinP'], axis=1) 

# Make a y2 based upon Winning Leage Series, for possible second model.B
y = teams_num['WinP']

In [15]:
from sklearn.neural_network import MLPRegressor

In [29]:
reg = MLPRegressor(hidden_layer_sizes=(250,), early_stopping=True)
reg.fit(X, y)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=(250,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [30]:
reg.score(X_old, y_old)

# Not a good model...

-39304.549853229015